In [1]:
%cd /content/drive/MyDrive/Cifar10

/content/drive/MyDrive/Cifar10


In [ ]:
import torch
import torch.nn.functional as f
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from tqdm import tqdm

ネットワークモデル

In [ ]:
class MyCNN(torch.nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(3,  # チャネル入力
                                     6,  # チャンネル出力
                                     5,  # カーネルサイズ
                                     1,  # ストライド (デフォルトは1)
                                     0,  # パディング (デフォルトは0)
                                     )
        self.conv2 = torch.nn.Conv2d(6, 16, 5)
 
        self.pool = torch.nn.MaxPool2d(2, 2)  # カーネルサイズ, ストライド
 
        self.dropout1 = torch.nn.Dropout2d(p=0.3)  # [new] Dropoutを追加してみる
 
        self.fc1 = torch.nn.Linear(16 * 5 * 5, 120)  # 入力サイズ, 出力サイズ
        self.dropout2 = torch.nn.Dropout(p=0.5)  # [new] Dropoutを追加してみる
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)
 
    def forward(self, x):
        x = f.relu(self.conv1(x))
        x = self.pool(x)
        x = f.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout1(x)  # [new] Dropoutを追加
        x = x.view(-1, 16 * 5 * 5)  # 1次元データに変えて全結合層へ
        x = f.relu(self.fc1(x))
        x = self.dropout2(x)   # [new] Dropoutを追加
        x = f.relu(self.fc2(x))
        x = self.fc3(x)
 
        return x
 
    # 畳み込みカーネルの表示
    def plot_conv1(self, prefix_num=0):
        weights1 = self.conv1.weight
        weights1 = weights1.reshape(3*6, 5, 5)
 
        for i, weight in enumerate(weights1):
            plt.subplot(3, 6, i + 1)
            plt.imshow(weight.data.to('cpu').numpy(), cmap='winter')
            plt.tick_params(labelbottom=False,
                            labelleft=False,
                            labelright=False,
                            labeltop=False,
                            bottom=False,
                            left=False,
                            right=False,
                            top=False)
 
        plt.savefig('img/{}_conv1.png'.format(prefix_num))
        plt.close()
 
    def plot_conv2(self, prefix_num=0):
        weights2 = self.conv2.weight
        weights2 = weights2.reshape(6*16, 5, 5)
 
        for i, weight in enumerate(weights2):
            plt.subplot(6, 16, i + 1)
            plt.imshow(weight.data.to('cpu').numpy(), cmap='winter')
            plt.tick_params(labelbottom=False,
                            labelleft=False,
                            labelright=False,
                            labeltop=False,
                            bottom=False,
                            left=False,
                            right=False,
                            top=False)
 
        plt.savefig('img/{}_conv2.png'.format(prefix_num))
        plt.close()
 

データローダー

In [ ]:
def load_cifar10(batch=128):
    train_loader = DataLoader(
        datasets.CIFAR10('./data',
                         train=True,
                         download=True,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(
                                [0.5, 0.5, 0.5],  # RGB 平均
                                [0.5, 0.5, 0.5]   # RGB 標準偏差
                                )
                         ])),
        batch_size=batch,
        shuffle=True
    )
 
    test_loader = DataLoader(
        datasets.CIFAR10('./data',
                         train=False,
                         download=True,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(
                                 [0.5, 0.5, 0.5],  # RGB 平均
                                 [0.5, 0.5, 0.5]  # RGB 標準偏差
                             )
                         ])),
        batch_size=batch,
        shuffle=True
    )
 
    return {'train': train_loader, 'test': test_loader}

学習とテストの実行

In [ ]:
 #=====================================================================
if __name__ == '__main__':
    epoch = 300
 
    loader = load_cifar10()
    classes = ('plane', 'car', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck')
 
    net: MyCNN = MyCNN()
    criterion = torch.nn.CrossEntropyLoss()  # ロスの計算
    optimizer = torch.optim.SGD(params=net.parameters(), lr=0.001, momentum=0.9)
 
    # もしGPUが使えるなら使う
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    net.to(device)
    print(device)
 
    # 学習前のフィルタの可視化
    net.plot_conv1()
    net.plot_conv2()
 
    history = {
        'train_loss': [],
        'train_acc': [],
        'test_acc': []
    }
 
    for e in range(epoch):
        net.train()
        loss = None
        for i, (images, labels) in enumerate(loader['train']):
            images = images.to(device)  # to GPU?
            labels = labels.to(device)
 
            optimizer.zero_grad()
            output = net(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
 
            if i % 10 == 0:
                print('Training log: {} epoch ({} / 50000 train. data). Loss: {}'.format(e + 1,
                                                                                         (i + 1) * 128,
                                                                                         loss.item())
                      )
 
        # 学習過程でのフィルタの可視化
        # net.plot_conv1(e+1)
        # net.plot_conv2(e+1)
 
        history['train_loss'].append(loss.item())
 
        net.eval()
        correct = 0
        with torch.no_grad():
            for i, (images, labels) in enumerate(tqdm(loader['train'])):
                images = images.to(device)  # to GPU?
                labels = labels.to(device)
 
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                correct += (predicted == labels).sum().item()
 
        acc = float(correct / 50000)
        history['train_acc'].append(acc)
 
        correct = 0
        with torch.no_grad():
            for i, (images, labels) in enumerate(tqdm(loader['test'])):
                images = images.to(device)  # to GPU?
                labels = labels.to(device)
 
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                correct += (predicted == labels).sum().item()
 
        acc = float(correct / 10000)
        history['test_acc'].append(acc)
 

        print("Accuracy : %f" % acc)

    # 学習前のフィルタの可視化
    net.plot_conv1(300)
    net.plot_conv2(300)
 
    # 結果をプロット
    plt.plot(range(1, epoch+1), history['train_loss'])
    plt.title('Training Loss [CIFAR10]')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.savefig('img/cifar10_loss.png')
    plt.close()
 
    plt.plot(range(1, epoch + 1), history['train_acc'], label='train_acc')
    plt.plot(range(1, epoch + 1), history['test_acc'], label='test_acc')
    plt.title('Accuracies [CIFAR10]')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend()
    plt.savefig('img/cifar10_acc.png')
    plt.close()

In [ ]:
!pwd

/content


In [ ]:
!ls

cifar10_cnn_003.ipynb  data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [11]:
import numpy as np

In [12]:
a=np.array([1])
print(a)

[1]


In [13]:
b=a
print(b)

[1]


In [14]:
b=[2]
print(b)

[2]


In [15]:
print(a,b)

[1] [2]
